In [1]:
from moviepy.editor import VideoFileClip

# Path to your video file
video_path = "AI_101.mp4"

# Create a video clip object
clip = VideoFileClip(video_path)

clip = clip.subclip(127, 427) 

# Extract audio and save as mp3
clip.audio.write_audiofile("AI_101(5min).mp3")


MoviePy - Writing audio in AI_101(5min).mp3


MoviePy - Done.


### Generate transcripts with timestamps

In [1]:
import whisper_timestamped as whisper

model = whisper.load_model("medium", device="cuda")

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
audio = whisper.load_audio("../../../AI_101(5min).mp3")

In [3]:
result = whisper.transcribe(model, audio, language="en")

100%|██████████| 30000/30000 [00:54<00:00, 549.90frames/s]


In [4]:
result

{'text': " the main takeaway, which is with the right data and the right model, artificial intelligence can solve many problems, but choosing the right problem, finding the right data and training the right model can be difficult. So, there's nothing else that you remember from today's workshop. This slide is the key takeaway. So, like I said, we'll cover artificial intelligence, we will see a few different examples of tools that can be used to implement artificial intelligence including something called machine learning. We'll talk about different kinds of machine learning and different problems that can be solved by machine learning. And let me just say since probably a lot of you are already wondering, what is deep learning deep learning is either a subset or a subset of a subset of machine learning. So for those of you who are familiar with neural networks or deep learning that is a tool that might be used in machine learning problems, we won't talk about that in any depth at all t

In [5]:
result["text"]

" the main takeaway, which is with the right data and the right model, artificial intelligence can solve many problems, but choosing the right problem, finding the right data and training the right model can be difficult. So, there's nothing else that you remember from today's workshop. This slide is the key takeaway. So, like I said, we'll cover artificial intelligence, we will see a few different examples of tools that can be used to implement artificial intelligence including something called machine learning. We'll talk about different kinds of machine learning and different problems that can be solved by machine learning. And let me just say since probably a lot of you are already wondering, what is deep learning deep learning is either a subset or a subset of a subset of machine learning. So for those of you who are familiar with neural networks or deep learning that is a tool that might be used in machine learning problems, we won't talk about that in any depth at all today but 

In [6]:
text = result["text"]

In [7]:
import gc
import torch

del model
gc.collect()
torch.cuda.empty_cache()

### Generate summary and sections

In [8]:
from llama_cpp import Llama

class LlamaCPP():
  def __init__(self, model_dir: str, **kwargs):
    print(f"Loading model: {model_dir}...")
    self.model_dir = model_dir
    self.llm = Llama(
      model_path=self.model_dir,
      n_gpu_layers=-1,
      seed=1234,
      n_ctx=4096,
    )

    defaults = {
      "temperature": 1.0,
      "repeat_penalty": 1.025,
      "min_p": 0.5,
      "top_p": 1.0,
      "top_k": 0,
      "max_tokens": 4096,
    }
    defaults.update(kwargs)
    self.args = defaults

      
  def generate(self, prompt: str, **kwargs):
    llm_args = self.args
    llm_args.update(kwargs)
    print("Prompt: ", prompt)
    print("Args: ", llm_args)
    output = self.llm(
      prompt,
      echo=False,
      **llm_args
    )
    print(f"output: {output}")
    response = output["choices"][0]["text"]

    return response

In [9]:
llm = LlamaCPP(model_dir=f"../video-summarizer-backend/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf")

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from ../video-summarizer-backend/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv

Loading model: ../video-summarizer-backend/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf...


llm_load_vocab: special tokens cache size = 256
llm_load_vocab: token to piece cache size = 0.8000 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = BPE
llm_load_print_meta: n_vocab          = 128256
llm_load_print_meta: n_merges         = 280147
llm_load_print_meta: n_ctx_train      = 8192
llm_load_print_meta: n_embd           = 4096
llm_load_print_meta: n_head           = 32
llm_load_print_meta: n_head_kv        = 8
llm_load_print_meta: n_layer          = 32
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_embd_head_k    = 128
llm_load_print_meta: n_embd_head_v    = 128
llm_load_print_meta: n_gqa            = 4
llm_load_print_meta: n_embd_k_gqa     = 1024
llm_load_print_meta: n_embd_v_gqa     = 1024
llm_load_print_meta: f_norm_eps       = 0.0e+00
llm_load_print_meta: f_norm_rms_eps   = 1.0e-05
llm_load_print_meta: f_clamp_kqv      = 0.0e+00
llm_load_print_meta: f_max_alib

In [10]:
prompt = f"""<|start_header_id|>user<|end_header_id|>
   
<instruction>Summarise the following transcript: <transcript>{text}</transcript></instruction><|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>

<summary>"""

In [11]:
summary = llm.generate(prompt)

Prompt:  <|start_header_id|>user<|end_header_id|>
   
<instruction>Summarise the following transcript: <transcript> the main takeaway, which is with the right data and the right model, artificial intelligence can solve many problems, but choosing the right problem, finding the right data and training the right model can be difficult. So, there's nothing else that you remember from today's workshop. This slide is the key takeaway. So, like I said, we'll cover artificial intelligence, we will see a few different examples of tools that can be used to implement artificial intelligence including something called machine learning. We'll talk about different kinds of machine learning and different problems that can be solved by machine learning. And let me just say since probably a lot of you are already wondering, what is deep learning deep learning is either a subset or a subset of a subset of machine learning. So for those of you who are familiar with neural networks or deep learning that 


llama_print_timings:        load time =     572.66 ms
llama_print_timings:      sample time =     953.83 ms /   138 runs   (    6.91 ms per token,   144.68 tokens per second)
llama_print_timings: prompt eval time =    1088.99 ms /  1045 tokens (    1.04 ms per token,   959.61 tokens per second)
llama_print_timings:        eval time =    3849.43 ms /   137 runs   (   28.10 ms per token,    35.59 tokens per second)
llama_print_timings:       total time =    6055.55 ms /  1182 tokens


output: {'id': 'cmpl-41bd90db-1755-4b29-96ff-ead0c397318b', 'object': 'text_completion', 'created': 1720274340, 'model': '../video-summarizer-backend/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf', 'choices': [{'text': ' The main takeaway from the workshop is that artificial intelligence (AI) can solve many problems, but choosing the right problem, finding the right data, and training the right model can be difficult. The speaker emphasizes that most AI today is narrow AI, which means it can only perform a specific task, such as self-driving cars or language processing. Narrow AI is not general AI, which would allow machines to learn any task that humans can perform. The speaker also discusses the basics of AI, including machine learning, deep learning, and expert systems. They highlight the challenges of defining rules for complex problems, such as identifying a table, and suggest that there are many different ways to implement AI.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'

In [12]:
print(summary)

 The main takeaway from the workshop is that artificial intelligence (AI) can solve many problems, but choosing the right problem, finding the right data, and training the right model can be difficult. The speaker emphasizes that most AI today is narrow AI, which means it can only perform a specific task, such as self-driving cars or language processing. Narrow AI is not general AI, which would allow machines to learn any task that humans can perform. The speaker also discusses the basics of AI, including machine learning, deep learning, and expert systems. They highlight the challenges of defining rules for complex problems, such as identifying a table, and suggest that there are many different ways to implement AI.


In [13]:
prompt = f"""<|start_header_id|>user<|end_header_id|>
   
<instruction>Split the following transcript into suitable sections verbatim with this: <section></section>""
Text: <transcript>{text}</transcript></instruction><|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>

<section>"""

In [14]:
sections_str = llm.generate(prompt)

Llama.generate: prefix-match hit


Prompt:  <|start_header_id|>user<|end_header_id|>
   
<instruction>Split the following transcript into suitable sections verbatim with this: <section></section>""
Text: <transcript> the main takeaway, which is with the right data and the right model, artificial intelligence can solve many problems, but choosing the right problem, finding the right data and training the right model can be difficult. So, there's nothing else that you remember from today's workshop. This slide is the key takeaway. So, like I said, we'll cover artificial intelligence, we will see a few different examples of tools that can be used to implement artificial intelligence including something called machine learning. We'll talk about different kinds of machine learning and different problems that can be solved by machine learning. And let me just say since probably a lot of you are already wondering, what is deep learning deep learning is either a subset or a subset of a subset of machine learning. So for those o


llama_print_timings:        load time =     572.66 ms
llama_print_timings:      sample time =    7092.34 ms /  1042 runs   (    6.81 ms per token,   146.92 tokens per second)
llama_print_timings: prompt eval time =     913.15 ms /  1049 tokens (    0.87 ms per token,  1148.76 tokens per second)
llama_print_timings:        eval time =   26748.73 ms /  1041 runs   (   25.70 ms per token,    38.92 tokens per second)
llama_print_timings:       total time =   36560.32 ms /  2090 tokens


output: {'id': 'cmpl-005b32ef-cab9-4da1-b97b-f901cab4775a', 'object': 'text_completion', 'created': 1720274346, 'model': '../video-summarizer-backend/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf', 'choices': [{'text': "the main takeaway, which is with the right data and the right model, artificial intelligence can solve many problems, but choosing the right problem, finding the right data and training the right model can be difficult. So, there's nothing else that you remember from today's workshop. This slide is the key takeaway.</section>\n\n<section>So, like I said, we'll cover artificial intelligence, we will see a few different examples of tools that can be used to implement artificial intelligence including something called machine learning. We'll talk about different kinds of machine learning and different problems that can be solved by machine learning. And let me just say since probably a lot of you are already wondering, what is deep learning deep learning is either a subset o

In [15]:
print(sections_str)

the main takeaway, which is with the right data and the right model, artificial intelligence can solve many problems, but choosing the right problem, finding the right data and training the right model can be difficult. So, there's nothing else that you remember from today's workshop. This slide is the key takeaway.</section>

<section>So, like I said, we'll cover artificial intelligence, we will see a few different examples of tools that can be used to implement artificial intelligence including something called machine learning. We'll talk about different kinds of machine learning and different problems that can be solved by machine learning. And let me just say since probably a lot of you are already wondering, what is deep learning deep learning is either a subset or a subset of a subset of machine learning. So for those of you who are familiar with neural networks or deep learning that is a tool that might be used in machine learning problems, we won't talk about that in any depth

In [16]:
sections = sections_str.split("</section>")

In [17]:
sections = [section.replace("<section>", "") for section in sections]
sections = [section.replace("\n", "") for section in sections]
sections = [section for section in sections if not section.isspace() and not section == ""]

In [18]:
print(sections)

["the main takeaway, which is with the right data and the right model, artificial intelligence can solve many problems, but choosing the right problem, finding the right data and training the right model can be difficult. So, there's nothing else that you remember from today's workshop. This slide is the key takeaway.", "So, like I said, we'll cover artificial intelligence, we will see a few different examples of tools that can be used to implement artificial intelligence including something called machine learning. We'll talk about different kinds of machine learning and different problems that can be solved by machine learning. And let me just say since probably a lot of you are already wondering, what is deep learning deep learning is either a subset or a subset of a subset of machine learning. So for those of you who are familiar with neural networks or deep learning that is a tool that might be used in machine learning problems, we won't talk about that in any depth at all today b

In [19]:
result.keys()

dict_keys(['text', 'segments', 'language'])

In [20]:
result["segments"]

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 13.14,
  'text': ' the main takeaway, which is with the right data and the right model, artificial intelligence can solve many problems, but choosing the right problem, finding the right data and training the right model can be difficult.',
  'tokens': [50364,
   264,
   2135,
   30681,
   11,
   597,
   307,
   365,
   264,
   558,
   1412,
   293,
   264,
   558,
   2316,
   11,
   11677,
   7599,
   393,
   5039,
   867,
   2740,
   11,
   457,
   10875,
   264,
   558,
   1154,
   11,
   5006,
   264,
   558,
   1412,
   293,
   3097,
   264,
   558,
   2316,
   393,
   312,
   2252,
   13,
   51064],
  'temperature': 0.0,
  'avg_logprob': -0.11746751240321568,
  'compression_ratio': 1.6772486772486772,
  'no_speech_prob': 0.027983587235212326,
  'confidence': 0.871,
  'words': [{'text': 'the', 'start': 0.0, 'end': 0.06, 'confidence': 0.097},
   {'text': 'main', 'start': 0.06, 'end': 0.32, 'confidence': 0.9},
   {'text': 'takeaway,',

In [26]:
import string

def generate_timestamps(sections, word_data):
    timestamps = []
    word_index = 0
    words_time = []
    for segment in word_data["segments"]:
        words_time.extend(segment["words"])
    
    for section in sections:
        # Generate summaries for each section
        summarize_short_prompt = f"""<|start_header_id|>user<|end_header_id|>
   
<instruction>Summarise the following transcript into a few words: <transcript>{section}</transcript></instruction><|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>

<summary>"""
        summary_short = llm.generate(summarize_short_prompt)
        summary_short = summary_short.replace("</summary>", "")

        # Generate summaries for each section
        summarize_full_prompt = f"""<|start_header_id|>user<|end_header_id|>
   
<instruction>Summarise the following transcript: <transcript>{section}</transcript></instruction><|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>

<summary>"""
        summary_full = llm.generate(summarize_full_prompt)
        summary_full = summary_full.replace("</summary>", "")

        # Get start and end times for each section
        words = section.split()
        section_start = None
        section_end = None
        
        for word in words:
            while word_index < len(words_time):
                current_word = words_time[word_index]
                if current_word['text'].lower().strip(string.punctuation) == word.lower().strip(string.punctuation):
                    if section_start is None:
                        section_start = current_word['start']
                    section_end = current_word['end']
                    word_index += 1
                    break
                word_index += 1
        
        if section_start is not None and section_end is not None:
            timestamps.append({"timestamps": (section_start, section_end), "text": section, "summary_short": summary_short, "summary_full": summary_full})
        else:
            timestamps.append({"timestamps": (None, None), "text": section, "summary_short": summary_short, "summary_full": summary_full})
    
    return timestamps

In [27]:
sections_timestamps = generate_timestamps(sections, result)

Prompt:  <|start_header_id|>user<|end_header_id|>
   
<instruction>Summarise the following transcript into a few words: <transcript>the main takeaway, which is with the right data and the right model, artificial intelligence can solve many problems, but choosing the right problem, finding the right data and training the right model can be difficult. So, there's nothing else that you remember from today's workshop. This slide is the key takeaway.</transcript></instruction><|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>

<summary>
Args:  {'temperature': 1.0, 'repeat_penalty': 1.025, 'min_p': 0.5, 'top_p': 1.0, 'top_k': 0, 'max_tokens': 4096}


Llama.generate: prefix-match hit

llama_print_timings:        load time =     572.66 ms
llama_print_timings:      sample time =      70.27 ms /    11 runs   (    6.39 ms per token,   156.54 tokens per second)
llama_print_timings: prompt eval time =     108.91 ms /    85 tokens (    1.28 ms per token,   780.48 tokens per second)
llama_print_timings:        eval time =     162.01 ms /    10 runs   (   16.20 ms per token,    61.72 tokens per second)
llama_print_timings:       total time =     349.19 ms /    95 tokens
Llama.generate: prefix-match hit


output: {'id': 'cmpl-0aef0ad5-ad2b-4ce3-903c-99d87ec3930d', 'object': 'text_completion', 'created': 1720275326, 'model': '../video-summarizer-backend/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf', 'choices': [{'text': 'AI solves problems with right data/model</summary>', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 99, 'completion_tokens': 10, 'total_tokens': 109}}
Prompt:  <|start_header_id|>user<|end_header_id|>
   
<instruction>Summarise the following transcript: <transcript>the main takeaway, which is with the right data and the right model, artificial intelligence can solve many problems, but choosing the right problem, finding the right data and training the right model can be difficult. So, there's nothing else that you remember from today's workshop. This slide is the key takeaway.</transcript></instruction><|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>

<summary>
Args:  {'temperature': 1.0, 'repeat_penalty': 1.025, 'min_p': 0.5,


llama_print_timings:        load time =     572.66 ms
llama_print_timings:      sample time =     270.13 ms /    41 runs   (    6.59 ms per token,   151.78 tokens per second)
llama_print_timings: prompt eval time =     119.96 ms /    81 tokens (    1.48 ms per token,   675.22 tokens per second)
llama_print_timings:        eval time =     729.70 ms /    40 runs   (   18.24 ms per token,    54.82 tokens per second)
llama_print_timings:       total time =    1152.54 ms /   121 tokens
Llama.generate: prefix-match hit


output: {'id': 'cmpl-0e90d545-38d8-491e-b606-e77f5c73f7cf', 'object': 'text_completion', 'created': 1720275326, 'model': '../video-summarizer-backend/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf', 'choices': [{'text': ' The main takeaway from the workshop is that artificial intelligence (AI) can solve many problems when given the right data and model. However, finding the right problem, data, and model can be challenging.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 95, 'completion_tokens': 40, 'total_tokens': 135}}
Prompt:  <|start_header_id|>user<|end_header_id|>
   
<instruction>Summarise the following transcript into a few words: <transcript>So, like I said, we'll cover artificial intelligence, we will see a few different examples of tools that can be used to implement artificial intelligence including something called machine learning. We'll talk about different kinds of machine learning and different problems that can be solved by machine l

...

output: {'id': 'cmpl-6d500ee6-cad5-4463-b1de-630c8472b074', 'object': 'text_completion', 'created': 1720275338, 'model': '../video-summarizer-backend/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf', 'choices': [{'text': ' Narrow AI is particularly good at performing typical tasks that involve vision, language processing, and planning. Examples include:\n\n* Vision: understanding or creating images or videos\n* Language processing: understanding or creating speech or text (e.g. Siri, Amazon Alexa)\n* Planning: route planning, motion planning, task planning, and more\n\nThese tasks can be trained using algorithms, and are categorized as narrow AI tasks.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 148, 'completion_tokens': 82, 'total_tokens': 230}}
Prompt:  <|start_header_id|>user<|end_header_id|>
   
<instruction>Summarise the following transcript into a few words: <transcript>As far as how we actually build AI. There's a lot of different ways to cl


llama_print_timings:        load time =     572.66 ms
llama_print_timings:      sample time =      45.55 ms /     7 runs   (    6.51 ms per token,   153.67 tokens per second)
llama_print_timings: prompt eval time =     155.35 ms /   249 tokens (    0.62 ms per token,  1602.86 tokens per second)
llama_print_timings:        eval time =     106.20 ms /     6 runs   (   17.70 ms per token,    56.49 tokens per second)
llama_print_timings:       total time =     311.43 ms /   255 tokens
Llama.generate: prefix-match hit


output: {'id': 'cmpl-5167d379-dca5-41be-8029-e2051666c738', 'object': 'text_completion', 'created': 1720275340, 'model': '../video-summarizer-backend/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf', 'choices': [{'text': 'Expert systems limitations</summary>', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 263, 'completion_tokens': 6, 'total_tokens': 269}}
Prompt:  <|start_header_id|>user<|end_header_id|>
   
<instruction>Summarise the following transcript: <transcript>As far as how we actually build AI. There's a lot of different ways to classical ways to implement artificial intelligence. So, the things that we're learning about this and being like wow this is not intelligent at all this is super dumb are expert systems and tree search. So in an expert system, it's just a person writing a list of rules, if this than that, if this than that. This can be super useful for certain problems. But a lot of the things that we actually think of as really strai


llama_print_timings:        load time =     572.66 ms
llama_print_timings:      sample time =     595.96 ms /    92 runs   (    6.48 ms per token,   154.37 tokens per second)
llama_print_timings: prompt eval time =     155.94 ms /   245 tokens (    0.64 ms per token,  1571.12 tokens per second)
llama_print_timings:        eval time =    1563.89 ms /    91 runs   (   17.19 ms per token,    58.19 tokens per second)
llama_print_timings:       total time =    2385.00 ms /   336 tokens


output: {'id': 'cmpl-42e7e46f-359d-48bd-b5e2-35d250e7a3c7', 'object': 'text_completion', 'created': 1720275340, 'model': '../video-summarizer-backend/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf', 'choices': [{'text': ' The speaker discusses the limitations of classical AI methods, such as expert systems and tree search. Expert systems rely on humans writing rules (if this, then that) to solve problems, but this approach can be challenging, especially for complex problems. For example, defining what a table is becomes difficult when trying to write a list of rules. The speaker suggests that these methods can still be useful for certain problems, but are not always effective for more complex tasks.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 259, 'completion_tokens': 91, 'total_tokens': 350}}


In [28]:
print(sections_timestamps)

[{'timestamps': (0.0, 18.42), 'text': "the main takeaway, which is with the right data and the right model, artificial intelligence can solve many problems, but choosing the right problem, finding the right data and training the right model can be difficult. So, there's nothing else that you remember from today's workshop. This slide is the key takeaway.", 'summary_short': 'AI solves problems with right data/model', 'summary_full': ' The main takeaway from the workshop is that artificial intelligence (AI) can solve many problems when given the right data and model. However, finding the right problem, data, and model can be challenging.'}, {'timestamps': (21.0, 65.28), 'text': "So, like I said, we'll cover artificial intelligence, we will see a few different examples of tools that can be used to implement artificial intelligence including something called machine learning. We'll talk about different kinds of machine learning and different problems that can be solved by machine learning.

In [29]:
print(sections_timestamps[0]["summary_short"])

AI solves problems with right data/model


In [30]:
print(sections_timestamps[0]["summary_full"])

 The main takeaway from the workshop is that artificial intelligence (AI) can solve many problems when given the right data and model. However, finding the right problem, data, and model can be challenging.


### Captioning

In [39]:
from Katna.video import Video
from Katna.writer import KeyFrameDiskWriter
import os

# For windows, the below if condition is must.
if __name__ == "__main__":

  # initialize video module
  vd = Video()

  # number of images to be returned
  no_of_frames_to_returned = 12

  # initialize diskwriter to save data at desired location
  diskwriter = KeyFrameDiskWriter(location="selectedframes")

  # Video file path
  video_file_path = "../../../AI_101(5 mins).mp4"

  print(f"Input video file path = {video_file_path}")

  # extract keyframes and process data with diskwriter
  vd.extract_video_keyframes(
       no_of_frames=no_of_frames_to_returned, file_path=video_file_path,
       writer=diskwriter
  )

Input video file path = ../../../AI_101(5 mins).mp4
Completed processing for :  ../../../AI_101(5 mins).mp4


In [40]:
from PIL import Image
import os
from transformers import AutoProcessor, AutoModelForCausalLM 


model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True)

config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

e:\work\Python_venv\RAG-chat\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\waiho\.cache\huggingface\hub\models--microsoft--Florence-2-large-ft. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

processing_florence2.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large-ft:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
C:\Users\waiho\.cache\huggingface\modules\transformers_modules\microsoft\Florence-2-large-ft\3112cd2e25c969cfdcb600a01489c56737d943d3\processing_florence2.py:499: SyntaxWarning: invalid escape sequence '\d'
  """


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [45]:
prompt = "<MORE_DETAILED_CAPTION>"

image = Image.open("selectedframes/AI_101(5 mins)_0.jpeg")

inputs = processor(text=prompt, images=image, return_tensors="pt")

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    do_sample=False,
    num_beams=3
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task="<MORE_DETAILED_CAPTION>", image_size=(image.width, image.height))

print(parsed_answer["<MORE_DETAILED_CAPTION>"])


A small white car is driving on the street. There is a large field of grass behind the car. There are poles in front of the field. 


In [53]:
captions = []

for image_path in os.listdir("selectedframes"):
    image = Image.open(f"selectedframes/{image_path}")

    inputs = processor(text=prompt, images=image, return_tensors="pt")

    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        do_sample=False,
        num_beams=3
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

    parsed_answer = processor.post_process_generation(generated_text, task="<MORE_DETAILED_CAPTION>", image_size=(image.width, image.height))

    captions.append(parsed_answer["<MORE_DETAILED_CAPTION>"])

In [54]:
captions_str = ""
for caption in captions:
    captions_str += "<caption>" + caption + "</caption>"

# Generate summary for captions
summarize_captions_prompt = f"""<|start_header_id|>user<|end_header_id|>

<instruction>Given some captions for key frames extracted from a video, summarise the video: <captions>{captions_str}</captions></instruction><|eot_id|> 
<|start_header_id|>assistant<|end_header_id|>

<summary>"""
summary_captions = llm.generate(summarize_captions_prompt)

Prompt:  <|start_header_id|>user<|end_header_id|>

<instruction>Given some captions for key frames extracted from a video, summarise the video: <captions><caption>A small white car is driving on the street. There is a large field of grass behind the car. There are poles in front of the field. </caption><caption>A robot head is shown in a black and white photo. The robot has a large head with two eyes and a mouth. There are wires coming out of the head. There is smoke behind the robot.</caption><caption>On the left side of the image there is a robot head. The image is in black and white. On the right side the image is of a small white car. The car is in a parking lot. The parking lot has yellow lines on it. There is a field of grass behind the car.</caption><caption>A small white car is driving on the street. There is a large field of grass behind the car. There are poles in the field. </caption><caption>On the left side of the image there is a robot head. The image is in black and whit

Llama.generate: prefix-match hit

llama_print_timings:        load time =     405.48 ms
llama_print_timings:      sample time =    1505.87 ms /   230 runs   (    6.55 ms per token,   152.74 tokens per second)
llama_print_timings: prompt eval time =     292.85 ms /   436 tokens (    0.67 ms per token,  1488.80 tokens per second)
llama_print_timings:        eval time =    4494.62 ms /   229 runs   (   19.63 ms per token,    50.95 tokens per second)
llama_print_timings:       total time =    6522.34 ms /   665 tokens


output: {'id': 'cmpl-ff1a06e3-2d4a-4245-8eb1-f9e300cce102', 'object': 'text_completion', 'created': 1720266253, 'model': '../video-summarizer-backend/models/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf', 'choices': [{'text': 'Based on the captions, the video appears to be a collection of scenes featuring a small white car and a robot head. The video starts with the car driving on a street with a large field of grass behind it, with poles in front of the field. The car is then shown parked in a parking lot with yellow lines, and the field of grass is visible behind it.\n\nThe robot head is introduced in a black and white photo, with wires coming out of its head and smoke behind it. The robot head is also shown in other scenes, sometimes with glowing red eyes, and sometimes with a gray background.\n\nThe video also features a comparison between the car and the robot head, with the two images side by side. The car is shown driving on the road, with yellow lines and green grass behind it, while th

In [55]:
print(summary_captions)

Based on the captions, the video appears to be a collection of scenes featuring a small white car and a robot head. The video starts with the car driving on a street with a large field of grass behind it, with poles in front of the field. The car is then shown parked in a parking lot with yellow lines, and the field of grass is visible behind it.

The robot head is introduced in a black and white photo, with wires coming out of its head and smoke behind it. The robot head is also shown in other scenes, sometimes with glowing red eyes, and sometimes with a gray background.

The video also features a comparison between the car and the robot head, with the two images side by side. The car is shown driving on the road, with yellow lines and green grass behind it, while the robot head is shown with its glowing red eyes and gray background.

The final scene shows the car driving on the street again, with the field of grass and poles in front of it. Overall, the video appears to be a collecti